In [114]:
import pandas as pd
from datetime import datetime

In [115]:
df = pd.read_csv("logs.csv", sep=",")

In [116]:
df["Door_open"] = df["Door"]=="open"
df["Door_a_little_open"] = df["Door"]=="a bit open"

df["t"] = pd.to_datetime(df["t"], format="mixed")

df["Alarm level"] = df["Alarm level"].map(lambda x: x[:1])
df["Alarm level"] = df["Alarm level"].astype(int)

In [124]:
filtered_df = df.loc[(df['t'] >= '2025-09-18') & (df['t'] < '2025-09-23')]

array([0, 1, 2])